In [32]:
import csv
import cv2
import numpy as np
import pandas as pd

In [33]:
# load results
state_labels = pd.read_csv('../../Thomas/data/snout_coords_hmmstates.csv')['states'].tolist()
coordinates_box = pd.read_csv('../data/cow_coords.csv', names=['x1', 'y2', 'w', 'h'])
coordinates_box = pd.DataFrame(coordinates_box)
print(coordinates_box['x1'][0])
# print(state_labels)

0.166457


In [34]:
# read video of the cow
video_path = '../data/cow_with_3-boxes.mp4'
video = cv2.VideoCapture(video_path)

# get video properties
fps = video.get(cv2.CAP_PROP_FPS)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))


# define the codec and create a VideoWriter object
codecc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('HMM_labeled_snout.mp4', codecc, fps, (width, height))

print(width)
print(height)

1458
786


In [35]:

# process each frame and add state labels
frame_count = 0
while True:
    ret, frame = video.read()
    if not ret:
        break  # video ends
    

    # Calculate the position for the text just under the rectangle
    text_x = (coordinates_box['x1'][frame_count]*100).astype(int)  # Adjust this based on the position of the rectangle
    text_y = ((coordinates_box['h'][frame_count] - coordinates_box['y2'][frame_count])*500).astype(int)     # Adjust the offset as needed
    # print(text_x, '  ', text_y)
    # print(coordinates_box['x1'][frame_count]*1000, '  ', coordinates_box['x1'][frame_count].astype(int)*1000)
    
    # Add state label to the frame just under the rectangle
    state_label = state_labels[frame_count] if frame_count < len(state_labels) else 'N/A'
    cv2.putText(frame, f'State {state_label}', (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write the frame to the output video
    output_video.write(frame)

    frame_count += 1

# Release video objects
video.release()
output_video.release()
cv2.destroyAllWindows()
